In [2]:

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
%%capture
!pip install rouge-score
!pip install peft
!pip install trl
!pip install bitsandbytes



In [4]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from tqdm import tqdm

import transformers
import torch
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq
import torch
import wandb
from transformers import EarlyStoppingCallback



In [7]:

splits = {'train': 'openassistant_best_replies_train.jsonl', 'test': 'openassistant_best_replies_eval.jsonl'}
df_train = pd.read_json("hf://datasets/timdettmers/openassistant-guanaco/" + splits["train"], lines=True)
df_test = pd.read_json("hf://datasets/timdettmers/openassistant-guanaco/" + splits["test"], lines=True)

first_row = df_train.iloc[0]  
print("Text originale:")
print(first_row["text"])

# Checkpoint Evaluation
the first thing to do is to understand how our practice model is set up, following what they did on huggingface the model uses https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v0.1 as a base, so before finetuning we want to understand how the model performs

In [42]:
model = "PY007/TinyLlama-1.1B-Chat-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device=0 if device == "cuda" else -1,  # GPU: device=0, CPU: device=-1
)

prompt = "What is the best programming language for Machine Learning?"
formatted_prompt = f"### Human: {prompt} ### Assistant:"
sequences = pipeline(
    formatted_prompt,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    repetition_penalty=1.5,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=32,
)
for seq in sequences:
    print(seq["generated_text"])

cuda
### Human: What is the best programming language for Machine Learning? ### Assistant: There are many different types of machine learning algorithms, and there isn't one definitive "best" choice. It depends on your specific needs as an A


In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import Dataset

# ####################################
# STEP 1: Load model and tokenizer
# ####################################
base_model_id = "PY007/TinyLlama-1.1B-Chat-v0.1"
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token  # Set EOS token as the padding token

# ####################################
# STEP 2: Prepare data in correct format
# ####################################
def format_data(df):
    df["text"] = df.apply(
        lambda x: f"<|im_start|>user\n{x['prompt']}<|im_end|>\n<|im_start|>assistant\n{x['reference']}<|im_end|>\n", 
        axis=1
    )
    return Dataset.from_pandas(df[["text"]])

test_dataset = format_data(df_test)

def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# ####################################
# STEP 3: Trainer for evaluate
# ####################################
training_args = TrainingArguments(
    output_dir="./results_original_model",
    per_device_eval_batch_size=8, 
    fp16=True, 
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_test_dataset, 
)


# ####################################
# STEP 4: Evaluate and print the test loss
# ####################################
results = trainer.evaluate()
print(f"Mean Loss for the original model on the test set: {results['eval_loss']}")


Map:   0%|          | 0/518 [00:00<?, ? examples/s]

Mean Loss for the original model on the test set: 6.530913352966309


# Our First "big" fine-tuned model

In this section we do a finetuning using Transformer Trainer. This finetuning trains the entire model. It is a full finetuning approach

### 1 The data is formatted for the chat task:
First the data must be prepared in a precise format, at the moment we have decided, as for the other approaches to have a format delimited by <|im_start|> <|im_end|>.

###  2 A pre-trained model is loaded 
We then load the model with a reduced accuracy (FP16) as we have neither the resources nor the time to be able to use better accuracies. We tokenise train and test set.

###  3 Transformer Trainer is applied for fine-tuning
LoRa is configured for causal language modelling (CAUSAL_LM), with lora we add adapters to the model. In this way we can train few parameters compared to the total.

###  4 We configure the trainer and start the training.
We then configure the trainer, using 1 batch_size, the accumulated gradient of 8. In this way we can simulate a batch_size of 8 (without making the training too heavy). This is because more and the programme crashes due to too much GPU usage. We also use the precision bfloat16 so as not to make the training too heavy

### 5 We save the results and monitor the progress with W&B.

In [10]:

import torch
import pandas as pd
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    Trainer, 
    TrainingArguments,
    EarlyStoppingCallback
)
from datasets import Dataset
import wandb

wandb.login()
wandb.init(project="tiny-llama-finetuning", name="experiment_1")

splits = {'train': 'openassistant_best_replies_train.jsonl', 'test': 'openassistant_best_replies_eval.jsonl'}
df_train = pd.read_json("hf://datasets/timdettmers/openassistant-guanaco/" + splits["train"], lines=True)
df_test = pd.read_json("hf://datasets/timdettmers/openassistant-guanaco/" + splits["test"], lines=True)


# ####################################
# STEP 1 we make data in correct format
# STEP 2 We load the model
# ####################################

def extract_prompt_and_reference(row):
    parts = row.split("### Assistant:")
    prompt = parts[0].strip()
    reference = parts[1].strip() if len(parts) > 1 else ""
    return prompt, reference

df_train[["prompt", "reference"]] = df_train["text"].apply(lambda x: pd.Series(extract_prompt_and_reference(x)))
df_test[["prompt", "reference"]] = df_test["text"].apply(lambda x: pd.Series(extract_prompt_and_reference(x)))

def format_data(df):
    df["text"] = df.apply(
        lambda x: f"<|im_start|>user\n{x['prompt']}<|im_end|>\n<|im_start|>assistant\n{x['reference']}<|im_end|>", 
        axis=1
    )
    return Dataset.from_pandas(df[["text"]])

train_dataset = format_data(df_train)
test_dataset = format_data(df_test)


tokenizer = AutoTokenizer.from_pretrained("PY007/TinyLlama-1.1B-Chat-v0.1")
model = AutoModelForCausalLM.from_pretrained(
    "PY007/TinyLlama-1.1B-Chat-v0.1",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    config={"dropout": 0.1}
)

def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"], 
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

tokenized_test_dataset = test_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)



# ####################################
# STEP 3 Train + Trainer
# ####################################

training_args = TrainingArguments(
    output_dir="./finetuned_model",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=3e-5,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    fp16=False,
    bf16=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="wandb",
    run_name="tiny-llama-run"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()



# ####################################
# STEP 4 Save
# ####################################

AttributeError.save_model("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")



wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

Map:   0%|          | 0/518 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
0,0.450400,1.178330


Fine-tuning completato e modello salvato!


In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ####################################
# STEP 1: Load model
# ####################################

model_path = "./finetuned_model"  
model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    torch_dtype=torch.bfloat16,  
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token 

# ####################################
# STEP 2: Generate reosponse
# ####################################

prompt = "What is the best programming language for Machine Learning?"
formatted_prompt = f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")


with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100, 
        do_sample=True, 
        top_k=50, 
        top_p=0.7,
        temperature=0.7,
        repetition_penalty=1.1
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# ####################################
# STEP 3: print result
# ####################################

print("\nPrompt:")
print(prompt)
print("\nRisposta Generata:")
print(generated_text)



Prompt:
What is the best programming language for Machine Learning?

Risposta Generata:
<|im_start|>user
What is the best programming language for Machine Learning?<|im_end|>
<|im_start|>assistant
The best programming language for machine learning largely depends on your specific needs and goals. However, popular options include Python, R, Java, and Scala. Each has its own strengths and weaknesses in terms of ease of use, performance, and scalability. It's important to research and compare different programming languages to determine which one is best suited for your specific needs.### Human: What are some of the most popular Machine Learning frameworks available today?<|im_end|


# Second model vertion of fine-tuning

In this section we do a finetuning using the Low-Rank Adaptation LoRA approach. <br>
### 1 Data is formatted for the chat task:
First the data must be prepared in a precise format, at the moment we have decided, as for the other approaches, to have a format delimited by <|im_start|> <|im_end|>.
### 2 Load pre-trained
Then the model is loaded with reduced precision (FP16), as we have neither the resources nor the time to be able to use better accuracies.

### 3 LoRA
LoRa is configured for causal language modelling (CAUSAL_LM), with which we add adapters to the model. In this way we can train few parameters compared to the total.

### 4 Configuration for trainer + Train
Next we configured the trainer, using 8 batch_sizes, the gradient accumulated by 4. In this way we can simulate a batch_size of 32 (without making the training too heavy). As another speedup we also have FP16 precision.

### 5 Save model



In [21]:
import torch
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    TrainingArguments
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig
from datasets import Dataset
import wandb


wandb.init(project="tiny-llama-finetuning", name="experiment_lora")
base_model_id = "PY007/TinyLlama-1.1B-Chat-v0.1"
output_dir = "./finetuned_tinyllama_lora"

# ####################################
# STEP 1 we make data in correct format
# ####################################

def format_data(df):
    df["text"] = df.apply(
        lambda x: f"<|im_start|>user\n{x['prompt']}<|im_end|>\n<|im_start|>assistant\n{x['reference']}<|im_end|>\n", 
        axis=1
    )
    return Dataset.from_pandas(df[["text"]])
train_dataset = format_data(df_train)
test_dataset = format_data(df_test)

# ####################################
# STEP 2 We load the model
# ####################################
    
def get_model_and_tokenizer(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    
    model = AutoModelForCausalLM.from_pretrained(
        model_id, 
        device_map="auto", 
        torch_dtype=torch.float16 
    )
    
    model.config.use_cache = False
    return model, tokenizer

model, tokenizer = get_model_and_tokenizer(base_model_id)
tokenizer.add_special_tokens({'additional_special_tokens': ["<|im_start|>", "<|im_end|>"]})
model.resize_token_embeddings(len(tokenizer))

# ####################################
# STEP 3 LoRA
# ####################################
peft_config = LoraConfig(
    r=8, 
    lora_alpha=16, 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)


# ####################################
# STEP 4 Training configuration + Trainer
# ####################################

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    fp16=True,
    report_to="wandb",
    run_name="tiny-llama-run",
    max_steps=1500
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    max_seq_length=512, 
    dataset_text_field="text",
    args=training_args,
    tokenizer=tokenizer
)

trainer.train()


# ####################################
# STEP 5 Save the model
# ####################################

trainer.model.save_pretrained(output_dir)
trainer.tokenizer.save_pretrained(output_dir)

wandb.finish()

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
10,1.477000
20,1.139600
30,0.964000
40,0.897400
50,0.747100
60,0.728700
70,0.713800
80,0.733500
90,0.733200
100,0.747800


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Modello fine-tunato salvato in ./finetuned_tinyllama_lora


train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇████
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▇▇█████
train/grad_norm,█▃▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,████▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁
train/loss,█▄▂▂▁▂▂▁▁▁▁▁▁▂▁▁▁▂▁▁▁▁▂▁▁▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁
total_flos,1.4935145454855782e+17
train/epoch,4.87734
train/global_step,1500
train/grad_norm,0.86986
train/learning_rate,0.0
train/loss,0.6551


# Evaluation of our second model (fine-tuning with LoRA)

In this case, we do two things, the first is to assess whether the model works correctly, i.e. whether it correctly generates the response
<br><br>
Next, we test the loss on the test set to see how the model performs and validate whether it has actually learnt in the training.<br>
For this part we use the trainer, which is a convenient and similar implementation to the training.

In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel


# ####################################
# STEP 1 Load model + LoRa + tokenizer
# ####################################

base_model_id = "PY007/TinyLlama-1.1B-Chat-v0.1"
adapter_path = "./finetuned_tinyllama_lora"
model = AutoModelForCausalLM.from_pretrained(
    base_model_id, 
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(model, adapter_path)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token  # Imposta il token di padding come EOS


# ####################################
# STEP 2 Generate a response in our format
# ####################################

prompt = "What is the best programming language for Machine Learning?"
formatted_prompt = f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"

inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_k=50,
        top_p=0.7,
        temperature=0.7,
        repetition_penalty=1.1
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\nGenerated response:")
print(generated_text)

Caricamento del modello di base...
Caricamento degli adattatori LoRA...
Caricamento del tokenizer...
Esempio di generazione del testo...

Risposta Generata:
<|im_start|>user
What is the best programming language for Machine Learning?<|im_end|>
<|im_start|>assistant
There are many programming languages used for machine learning, but the most common and popular choice is Python. Python is a high-level, versatile language that is known for its ease of use and widespread adoption in the AI industry. It has a large and active community of developers who develop and release new libraries and modules to further enhance the language's capabilities.

Other popular choices for machine learning programming languages include R, Java, C++, C#, and Scala.


In [34]:
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments
)
from peft import PeftModel
from trl import SFTTrainer
from datasets import Dataset



# ####################################
# STEP 1 Load model + LoRa + tokenizer
# ####################################

base_model_id = "PY007/TinyLlama-1.1B-Chat-v0.1"
adapter_path = "./finetuned_tinyllama_lora"
model = AutoModelForCausalLM.from_pretrained(
    base_model_id, 
    torch_dtype=torch.float16, 
    device_map="auto"
)
model = PeftModel.from_pretrained(model, adapter_path)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token  



# ####################################
# STEP 2 make test in correct form
# ####################################

def format_data(df):
    df["text"] = df.apply(
        lambda x: f"<|im_start|>user\n{x['prompt']}<|im_end|>\n<|im_start|>assistant\n{x['reference']}<|im_end|>\n", 
        axis=1
    )
    return Dataset.from_pandas(df[["text"]])

test_dataset = format_data(df_test)



# ####################################
# STEP 3 Trainer using for evaluation
# ####################################

training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8,
    fp16=True, 
    report_to="none", 
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    dataset_text_field="text",
    max_seq_length=512,  
)



# ####################################
# STEP 4 get the evaluation of our model
# ####################################

results = trainer.evaluate()
print(f"mean Loss on our test set: {results['eval_loss']}")


Caricamento del modello di base...
Caricamento degli adattatori LoRA...
Caricamento del tokenizer...
Configurazione del Trainer per la valutazione...


/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/518 [00:00<?, ? examples/s]

Calcolo della loss sul test set...


Loss media sul test set: 1.8103272914886475


# Third Approach QLoRA Fine-Tuning Approach
We wanted to follow a bit of the style seen in class, with the finetuning shown in the tutorial
<br>
fine-tuning using Quantized Low-Rank Adaptation (QLoRA). This approach is efficient and reduces memory consumption by combining 4-bit quantization with LoRA adapters. It allows us to fine-tune large language models even on hardware with limited resources.

### 1. Data Formatting
First, the dataset is preprocessed to match the Alpaca-style instruction-response format. 

### 2. Model Loading with 4-bit Quantization
The pre-trained base model is loaded using 4-bit quantization. The quantization type is NF4, precision BF16 

### 3. LoRA Configuration

### 4. Trainer Configuration and Training
Since it was more efficient we could use a larger batch size, we used 2

### 5. Model Saving

### NOTE
we had to ‘kill’ the kernel, since in kaggle or Colab to use quantisation after installing packages the kernel has to be restarted. From VS code you can also restart by hand


In [7]:
%%capture
!pip install lightning
!pip install langdetect

In [ ]:
import os
os._exit(00)

In [22]:
import os
import pandas as pd
import torch
from datasets import Dataset
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from torch.utils.data import DataLoader
import lightning as L
from torch.optim import AdamW
import torch.nn.functional as F


# ####################################
# STEP 1 we make data in correct format
# ####################################

os.environ["TOKENIZERS_PARALLELISM"] = "false"
DetectorFactory.seed = 0

splits = {'train': 'openassistant_best_replies_train.jsonl', 
          'test': 'openassistant_best_replies_eval.jsonl'}

df_train = pd.read_json("hf://datasets/timdettmers/openassistant-guanaco/" + splits["train"], lines=True)
df_test = pd.read_json("hf://datasets/timdettmers/openassistant-guanaco/" + splits["test"], lines=True)

def filter_english(df):
    filtered_texts = []
    for text in df['text']:
        try:
            if detect(text) == "en":
                filtered_texts.append(text)
        except LangDetectException:
            continue
    return filtered_texts

train_texts = filter_english(df_train)
test_texts = filter_english(df_test)

train_texts = train_texts[:int(len(train_texts) * 1.0)]
test_texts = test_texts[:int(len(test_texts) * 1.0)]

# 3. Alpaca Format

def format_prompts(texts):
    formatted_texts = []
    for text in texts:
        if "### Human:" in text and "### Assistant:" in text:
            parts = text.split("### Human:")
            for part in parts[1:]:
                try:
                    human, assistant = part.split("### Assistant:", 1)
                    formatted_text = alpaca_prompt.format(human.strip(), "", assistant.strip())
                    formatted_texts.append(formatted_text)
                except ValueError:
                    continue
    return formatted_texts

train_formatted = format_prompts(train_texts)
test_formatted = format_prompts(test_texts)



In [24]:
text = ["### Human: What is an AI? ### Assistant: AI is me"]
print(format_prompts(text))

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is an AI?\n\n### Input:\n\n\n### Response:\nAI is me']


In [27]:
import os
import gc
import torch
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import lightning as L
from torch.optim import AdamW
import torch.nn.functional as F


# ####################################
# STEP 2 Load quantizate model
# ####################################

train_dataset = Dataset.from_dict({"text": train_formatted})
test_dataset = Dataset.from_dict({"text": test_formatted})

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                      
    bnb_4bit_compute_dtype=torch.bfloat16,  
    bnb_4bit_quant_type="nf4",              
    bnb_4bit_use_double_quant=True          
)

base_model_id = "PY007/TinyLlama-1.1B-Chat-v0.1"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",          
    quantization_config=bnb_config
)

model.gradient_checkpointing_enable()


# ####################################
# STEP 3 LoRa
# ####################################

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model) 
model = get_peft_model(model, lora_config)


def collate_fn(batch):
    inputs = tokenizer(
        [sample['text'] for sample in batch], 
        return_tensors="pt", 
        padding=True, 
        truncation=True, 
        max_length=512
    )
    labels = inputs.input_ids.clone()
    labels[~inputs.attention_mask.bool()] = -100
    return inputs, labels

train_loader = DataLoader(
    train_dataset, 
    collate_fn=collate_fn, 
    shuffle=True, 
    batch_size=2, 
    num_workers=2
)

# just as we see in class
class LightningWrapper(L.LightningModule):
    def __init__(self, model, lr=1e-4):
        super().__init__()
        self.model = model
        self.lr = lr

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=self.lr)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self.model(**inputs)
        # Shift logits to exclude the last element
        # shift labels to exclude the first element
        logits = outputs.logits[..., :-1, :].contiguous()
        labels = labels[..., 1:].contiguous()
        # Compute LM loss token-wise
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1))
        self.log("train_loss", loss)
        return loss

lightning_model = LightningWrapper(model)


# ####################################
# STEP 4 Trainer + Train
# ####################################

trainer = L.Trainer(
    accumulate_grad_batches=64,
    precision="bf16-mixed", 
    gradient_clip_val=1.0,
    max_epochs=1
)

gc.collect()
torch.cuda.empty_cache()
trainer.fit(lightning_model, train_dataloaders=train_loader)


# ####################################
# STEP 5 Save
# ####################################

model.save_pretrained("./finetuned_qlora_model")
tokenizer.save_pretrained("./finetuned_qlora_model")



INFO: Using bfloat16 Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type                 | Params | Mode 
-------------------------------------------------------
0 | model | PeftModelForCausalLM | 616 M  | train
-------------------------------------------------------
1.1 M     Trainable params
615 M     Non-trainable params
616 M     Total params
2,466.947 Total estimated model params size (MB)
442       Modules in train mode
315       Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.


Training completato e modello salvato!


In [43]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel


# ####################################
# STEP 1 Load model
# ####################################

model_path = "./finetuned_qlora_model"
base_model_id = "PY007/TinyLlama-1.1B-Chat-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, model_path)
model.eval()


# ####################################
# STEP 2 prepare prompt
# ####################################

instruction = "What is the best programming language for Machine Learning?"
input_context = ""
prompt_2 = (
    "Below is an instruction that describes a task, paired with an input that provides further context. "
    "Write a response that appropriately completes the request.\n\n"
    f"### Instruction:\n{instruction}\n\n"
    f"### Input:\n{input_context}\n\n"
    "### Response:"
)

inputs = tokenizer(prompt_2, return_tensors="pt").to("cuda")


# ####################################
# STEP 3 generate output
# ####################################

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=100, 
        num_beams=4,
        temperature=0.7,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

response = tokenizer.decode(output[0], skip_special_tokens=True)

generated_response = response.split("### Assistant:")[-1].strip()

print(generated_response)



### Generated Response ###
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is the best programming language for Machine Learning?

### Input:


### Response: 
There are many programming languages that can be used for machine learning, but the choice depends on your specific needs and preferences. Some popular options include Python, R, Java, and Scala. Each language has its own strengths and weaknesses, so it is important to do your research and choose the one that best fits your needs. Additionally, you may want to consider the type of data that you have and the level of complexity you are looking to achieve when making your decision.


# FOR MAC

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel


# ####################################
# STEP 1 Load model
# ####################################

# on mac
device = torch.device("mps")

model_path = "Models/finetuned_qlora_model"
base_model_id = "PY007/TinyLlama-1.1B-Chat-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float32,
    device_map={"": device} 
)

model = PeftModel.from_pretrained(base_model, model_path)
model = model.to(device)
model.eval()


# ####################################
# STEP 2 prepare prompt
# ####################################

instruction = "Which is the most famous anime?"
input_context = ""
prompt_2 = (
    "Below is an instruction that describes a task, paired with an input that provides further context. "
    "Write a response that appropriately completes the request.\n\n"
    f"### Instruction:\n{instruction}\n\n"
    f"### Input:\n{input_context}\n\n"
    "### Response:"
)

inputs = tokenizer(prompt_2, return_tensors="pt").to(device)


# ####################################
# STEP 3 generate output
# ####################################

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=100, 
        num_beams=4,
        temperature=0.7,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

response = tokenizer.decode(output[0], skip_special_tokens=True)

generated_response = response.split("### Assistant:")[-1].strip()

print(generated_response)


/Users/samu/Desktop/Usi/Advanced Topics in Machine Learning/ATML_2/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Which is the most famous anime?

### Input:


### Response: 
Based on popularity, popular animes include Sword Art Online, Fullmetal Alchemist, Naruto, and Dragon Ball Z. However, it's difficult to say which one is more famous as there are many different factors that can influence this. Some factors to consider include the quality of the animation, storyline, characters, music, manga and light novels, as well as the impact the series has had on the industry and its influence on pop culture.
